In [6]:
import math
import numpy as np
import pandas as pd

from sklearn.model_selection import LeaveOneGroupOut
from sklearn.base import clone
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# --------------------------------------------------
# 1. Load data
# --------------------------------------------------
DATA_PATH = r"/Users/nima/Documents/Disk E/ML/DATA/Battery Aging/uncleaned Nasa Battery Data/discharge.csv"
df = pd.read_csv(DATA_PATH)

# Keep only discharge rows (if mixed)
df = df[df["type"] == "discharge"].copy()

# Basic cleanup: drop rows with missing key signals
needed = ["Battery", "id_cycle", "Voltage_measured", "Current_measured",
          "Temperature_measured", "ambient_temperature", "Time", "Capacity"]
df = df.dropna(subset=needed)

# --------------------------------------------------
# 2. Aggregate to cycle level
# --------------------------------------------------
group_cols = ["Battery", "id_cycle"]

agg_dict = {
    "Voltage_measured": ["mean", "std", "min", "max"],
    "Current_measured": ["mean", "std", "min", "max"],
    "Temperature_measured": ["mean", "std", "min", "max"],
    "ambient_temperature": ["mean"],
    "Time": ["min", "max"],
    "Capacity": ["last"],  # capacity at end of discharge
}

cycle_df = df.groupby(group_cols).agg(agg_dict)

# Flatten MultiIndex columns
cycle_df.columns = ["_".join([c for c in col if c]) for col in cycle_df.columns.to_flat_index()]
cycle_df = cycle_df.reset_index()

cycle_df["cycle_duration"] = cycle_df["Time_max"] - cycle_df["Time_min"]

# Optional: remove obviously broken cycles (duration <= 0)
cycle_df = cycle_df[cycle_df["cycle_duration"] > 0].copy()

# --------------------------------------------------
# 3. Define SOH per cycle (robust C0)
# --------------------------------------------------
# Robust C0: median of first 10 cycles (or fewer if battery has fewer cycles)
C0_dict = {}
for bat, sub in cycle_df.groupby("Battery"):
    early = sub.sort_values("id_cycle").head(10)
    C0 = float(np.median(early["Capacity_last"].values))
    C0_dict[bat] = C0

cycle_df["C0"] = cycle_df["Battery"].map(C0_dict)
cycle_df["SOH"] = cycle_df["Capacity_last"] / cycle_df["C0"] * 100.0

# --------------------------------------------------
# 4. Feature matrix (add id_cycle as a strong baseline predictor)
# --------------------------------------------------
cycle_df["log_cycle"] = np.log1p(cycle_df["id_cycle"].astype(float))

feature_cols = [
    "id_cycle",
    "log_cycle",

    "Voltage_measured_mean",
    "Voltage_measured_std",
    "Voltage_measured_min",
    "Voltage_measured_max",

    "Current_measured_mean",
    "Current_measured_std",
    "Current_measured_min",
    "Current_measured_max",

    "Temperature_measured_mean",
    "Temperature_measured_std",
    "Temperature_measured_min",
    "Temperature_measured_max",

    "ambient_temperature_mean",
    "cycle_duration",
]

X = cycle_df[feature_cols].values
y = cycle_df["SOH"].values
groups = cycle_df["Battery"].values

# --------------------------------------------------
# 5. Models (scale linear models)
# --------------------------------------------------
models = {
    "LinearRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LinearRegression())
    ]),
    "ElasticNet": Pipeline([
        ("scaler", StandardScaler()),
        ("model", ElasticNet(alpha=0.01, l1_ratio=0.5, random_state=0, max_iter=20000))
    ]),
    "RandomForest": RandomForestRegressor(
        n_estimators=400,
        random_state=0,
        n_jobs=-1,
        max_depth=None,
        min_samples_leaf=2
    ),
    "GradientBoosting": GradientBoostingRegressor(random_state=0),
    "ExtraTrees": ExtraTreesRegressor(
        n_estimators=400,
        random_state=0,
        n_jobs=-1,
        max_depth=None,
        min_samples_leaf=2
    ),
}

logo = LeaveOneGroupOut()

overall_raw = {name: {"y_true": [], "y_pred": []} for name in models}
per_battery_raw = {name: {} for name in models}

# --------------------------------------------------
# 6. Leave-One-Group-Out CV across batteries
# --------------------------------------------------
for train_idx, test_idx in logo.split(X, y, groups):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    test_battery = groups[test_idx][0]

    for name, model in models.items():
        m = clone(model)  # IMPORTANT: fresh estimator each fold
        m.fit(X_train, y_train)
        y_pred = m.predict(X_test)

        overall_raw[name]["y_true"].extend(y_test)
        overall_raw[name]["y_pred"].extend(y_pred)

        if test_battery not in per_battery_raw[name]:
            per_battery_raw[name][test_battery] = {"y_true": [], "y_pred": []}
        per_battery_raw[name][test_battery]["y_true"].extend(y_test)
        per_battery_raw[name][test_battery]["y_pred"].extend(y_pred)

# --------------------------------------------------
# 7. Metrics helper
# --------------------------------------------------
def compute_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

# --------------------------------------------------
# 8. Overall metrics
# --------------------------------------------------
overall_metrics = {}

print("=== Overall performance (Leave-One-Battery-Out) ===")
print("{:<18s} {:>10s} {:>10s} {:>8s}".format("Model", "MAE", "RMSE", "R2"))

for name, res in overall_raw.items():
    yt = np.array(res["y_true"])
    yp = np.array(res["y_pred"])
    mae, rmse, r2 = compute_metrics(yt, yp)
    overall_metrics[name] = {"mae": mae, "rmse": rmse, "r2": r2}
    print("{:<18s} {:10.3f} {:10.3f} {:8.3f}".format(name, mae, rmse, r2))

# --------------------------------------------------
# 9. Pick top 3 models by overall RMSE
# --------------------------------------------------
sorted_models = sorted(overall_metrics.items(), key=lambda kv: kv[1]["rmse"])
top3_names = [name for name, _ in sorted_models[:3]]

print("\n=== Top 3 models by overall RMSE ===")
for rank, name in enumerate(top3_names, start=1):
    m = overall_metrics[name]
    print(f"{rank}. {name:>16s}  MAE={m['mae']:.3f}, RMSE={m['rmse']:.3f}, R2={m['r2']:.3f}")

# --------------------------------------------------
# 10. Per-battery metrics for top 3
# --------------------------------------------------
print("\n=== Per-battery performance for Top 3 models (Leave-One-Battery-Out) ===")
batteries_sorted = sorted(cycle_df["Battery"].unique())

for name in top3_names:
    print(f"\n--- {name} ---")
    print("{:<8s} {:>10s} {:>10s} {:>8s}".format("Battery", "MAE", "RMSE", "R2"))

    for bat in batteries_sorted:
        res = per_battery_raw[name].get(bat, None)
        if res is None or len(res["y_true"]) == 0:
            continue
        yt = np.array(res["y_true"])
        yp = np.array(res["y_pred"])
        mae, rmse, r2 = compute_metrics(yt, yp)
        print("{:<8s} {:10.3f} {:10.3f} {:8.3f}".format(bat, mae, rmse, r2))


=== Overall performance (Leave-One-Battery-Out) ===
Model                     MAE       RMSE       R2
LinearRegression        5.275      6.278    0.674
ElasticNet              4.156      4.680    0.819
RandomForest            2.593      3.583    0.894
GradientBoosting        2.563      3.612    0.892
ExtraTrees              2.403      3.531    0.897

=== Top 3 models by overall RMSE ===
1.       ExtraTrees  MAE=2.403, RMSE=3.531, R2=0.897
2.     RandomForest  MAE=2.593, RMSE=3.583, R2=0.894
3. GradientBoosting  MAE=2.563, RMSE=3.612, R2=0.892

=== Per-battery performance for Top 3 models (Leave-One-Battery-Out) ===

--- ExtraTrees ---
Battery         MAE       RMSE       R2
B0005         1.264      1.730    0.972
B0006         5.565      6.205    0.753
B0007         1.316      1.946    0.948
B0018         1.212      1.559    0.966

--- RandomForest ---
Battery         MAE       RMSE       R2
B0005         1.784      2.006    0.962
B0006         5.242      6.152    0.757
B0007         1